### Задача регрессии, цель которой состоит в том, чтобы спрогнозировать площадь выгоревших лесных пожаров в северо-восточном регионе Португалии с использованием метеорологических и других данных.

#### Загрузка библтотек.

In [9]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils
from tensorflow.keras.backend import clear_session

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Загрузка данных.

In [2]:
x_train = np.load('../input/udt-3-regression/x_train.npy')
y_train = np.load('../input/udt-3-regression/y_train.npy')
x_test = np.load('../input/udt-3-regression/x_test.npy')

In [3]:
x_train.shape

#### Создаем нейронную сеть.

In [4]:
# Создаем автоматическое понижения скорости обучения модели.
reduce_lr = ReduceLROnPlateau(min_lr=0.000000001, verbose=0)

In [11]:
# Создание модели.
Inputs = keras.Input(shape=(12,))
x = Dense(128, activation=tf.keras.layers.LeakyReLU())(Inputs)
x = Dense(1000, activation=tf.keras.layers.LeakyReLU())(x)
x = Dense(100, activation=tf.keras.layers.LeakyReLU())(x)
outputs = Dense(1, activation='linear')(x)

model = keras.Model(inputs=Inputs, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=1e-3), loss=keras.losses.Huber(delta=0.085), metrics=['mae'])
model.summary()

In [12]:
# Обучение сети.
history = model.fit(x_train, 
                    y_train, 
                    epochs=100,
                    batch_size=10,
                    validation_split=0.2,
                    callbacks=[reduce_lr], 
                    verbose=1)


In [13]:
# Отображение изменения точности модели.
plt.figure(figsize=(10, 6))
plt.plot(history.history['mae'], 
         label='Средняя абсолютная ошибка на тренировочной выборке.')
plt.plot(history.history['val_mae'], 
         label='Средняя абсолютная ошибка на валидационной выборке.')
plt.xlabel('Эпоха обучения')
plt.ylabel('Средняя абсолютная ошибка')
plt.legend()
plt.show()

#### Проверка работы на тестовых данных.

In [14]:
# Предсказание на тестовых данных.
prediction = model.predict(x_test).flatten()
submission = pd.DataFrame({"Id":range(1,len(prediction)+1),"Label":prediction})

In [15]:
# Запись ответов в файл.
submission = submission.to_csv('./submission01_forest_file.csv',index=False,header=True)